In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626847 sha256=adc548de7f018b7a429dbe60d78abef292191391992042c9948391eea7b530e6
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
import surprise
gp = pd.read_csv('/content/drive/MyDrive/PJ/data/구매상품TR_reorder.csv',chunksize = 2000000)
gp_li = list(gp)
df=pd.concat(gp_li)
reorder_rate=df.groupby(['고객번호','소분류명'])[['reorder']].sum()
total_ordered_goods=df.groupby(['고객번호','소분류명'])[['reorder']].count().reset_index()
o=reorder_rate.reset_index()
o['rate'] = o['reorder']/total_ordered_goods['reorder']


en = LabelEncoder()
goods_name=en.fit_transform(o['소분류명'])

o['소분류명_encoded'] = goods_name
bins =pd.cut(o['rate'],5,labels = [1,2,3,4,5])
o['ratings'] = bins

o.columns = ['user_id','소분류명','reorder','rate','item_id','raiting']
o.info()
p = o[['user_id','item_id','raiting']].astype('int16')

reader = surprise.dataset.Reader(rating_scale = (0.5,5.0))
data=surprise.dataset.Dataset.load_from_df(p,reader)
trainset = data.build_full_trainset()

# param_grid={'n_epochs':[20,40,60],'n_factors':[50,100,200]}

# grid=GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
# grid.fit(trainset)

# print(grid.best_score['rmse'])
# print(grid.best_params['rmse'])
algo = surprise.SVD(n_epochs=60,n_factors=50)
try :
  algo.fit(trainset)
except:
  print(0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134917 entries, 0 to 6134916
Data columns (total 6 columns):
 #   Column   Dtype   
---  ------   -----   
 0   user_id  int64   
 1   소분류명     object  
 2   reorder  int64   
 3   rate     float64 
 4   item_id  int64   
 5   raiting  category
dtypes: category(1), float64(1), int64(3), object(1)
memory usage: 239.9+ MB


In [ ]:
pred = algo.predict(200000000,20000000)
pred

Prediction(uid=200000000, iid=20000000, r_ui=None, est=1.0617525876878204, details={'was_impossible': False})

In [ ]:
pred[3]

1.0499271271839357

In [ ]:
users = []
items = []
score = []

for i in range(1,19384):
    for j in range(0,3519):
        pred = algo.predict(i,j)
        users.append(pred[0])
        items.append(pred[1])
        score.append(pred[3])

d = {'users' : users, 'items' : items, 'score' : score}
df = pd.DataFrame(data = d)

In [ ]:
df[df.users==1].sort_values('score',ascending=False)[:5]

,users,items,score
1516,1,1516,2.052833
3233,1,3233,2.043997
405,1,405,1.970483
797,1,797,1.931274
1101,1,1101,1.916605
